In [6]:
import numpy as np
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Import SQL Data


The data for plotting surface temperature changes is loaded from an SQL file `Visualization_data_sql.db`, from which the table `Surface_Temperature_by_years` is selected with columns:

* `Month` - Month for which temperature observations were averaged.
* `mean_Surface_Temp` - Average surface temperature for the month over the specified altitude range.
* `Elevation` - Altitude range in meters above sea level.
* `year` - Year of observation.






In [ ]:
!wget 'https://www.dropbox.com/scl/fi/u7fuf81576hdjmq3ql5up/Visualization_data_sql.db?rlkey=qic0kyfv3tfs6qg4drjdfpwas&dl=0' -O Visualization_data_sql.db

In [13]:
%cd /content
conn = sqlite3.connect('Visualization_data_sql.db')
query = f"SELECT Month, mean_Surface_Temp, Elevation, year FROM Surface_Temperature_by_years"
data_year_line = pd.read_sql_query(query, conn)
conn.close()

/content


# Average Surface Temperature plot with slider by years.

This plot shows the values of the average surface temperature depending on the month of observation for the selected year. The values are differentiated by their altitude range.

In [15]:
fig = px.line(data_year_line, x="Month", y="mean_Surface_Temp", color='Elevation', markers=True, title='Average Surface Temperature',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f', '#042e00'], animation_frame="year")
fig.show()

# Average Surface Temperature plot with slider by months.

This plot shows the values of the average surface temperature depending on the year of observation for the selected month. The values are differentiated by their altitude range.

In [16]:
fig = px.line(data_year_line, x="year", y="mean_Surface_Temp", color='Elevation', markers=True, title='Average Surface Temperature',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f', '#042e00'], animation_frame="Month", range_y=(-40, 30))
fig.show()

# Multi Feature Plot

In this point, we will plot not only the average temperature values. We will show the minimum, mean, median, and quartile values of the surface temperature for each month from 2005 to the present. To do this, we load all the features from the Surface_Temperature_by_years table. Then we need to prepare the corresponding dataframe.

## import & preprocessing

In [3]:
%cd /content
conn = sqlite3.connect('Visualization_data_sql.db')
query = f"SELECT * FROM Surface_Temperature_by_years"
data_year_line_mf = pd.read_sql_query(query, conn)
conn.close()

/content


In [4]:
data_multi_feat = pd.DataFrame(columns = ['Month', 'Elevation', 'Surface_Temp', 'Aggr_func', 'year'])
col = ['mean_Surface_Temp', 'min_Surface_Temp', 'max_Surface_Temp', 'median_Surface_Temp', 'qu_25_Surface_Temp', 'qu_75_Surface_Temp']
func = ['Mean', 'Min', 'Max', 'Median', 'QU_25', 'QU_75']

for c, f in zip(col, func):
  df_temp = pd.DataFrame({'Month': data_year_line_mf.Month,
                          'Elevation': data_year_line_mf.Elevation,
                          'year': data_year_line_mf.year,
                          'Surface_Temp': data_year_line_mf[c],
                          'Aggr_func': f})
  data_multi_feat = pd.concat([data_multi_feat, df_temp], axis=0)

data_multi_feat['date'] = data_multi_feat.Month.astype(str) + ', ' + data_multi_feat.year.astype(str)
data_multi_feat.date = pd.to_datetime(data_multi_feat.date)
data_multi_feat = data_multi_feat.sort_values('date')
elev_type_order=['500-1000', '1000-2000', '2000-3000', '3000-4000', '4000-5000', '5000-6000', '6000-7000']
data_multi_feat['Elevation'] = pd.Categorical(data_multi_feat['Elevation'], categories=elev_type_order, ordered=True)
data_multi_feat = data_multi_feat.sort_values(by=['Elevation', 'date'])

## Plot

The plot shows the surface temperature value for the specified altitude range, aggregated by the following statistical functions:


* `Min` - Minimum surface temperature observed for the month
* `QU_25` - 25th percentile surface temperature observed for the month
* `Median` - Median surface temperature observed for the month
* `QU_75` - 75th percentile surface temperature observed for the month
* `Mean` - Mean (Average) surface temperature observed for the month
* `Max` - Maximum surface temperature observed for the month

In [5]:
fig = px.line(data_multi_feat, x="date", y="Surface_Temp", color='Aggr_func', markers=True, title='Surface Temperature',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f'], animation_frame="Elevation", range_y=(-50, 50))
fig.show()

In [7]:
years = data_multi_feat.year.unique().tolist()
def filter_dataframe(year_from: int, year_to: int)->None:
  global data_multi_feat_year
  data_multi_feat_year = data_multi_feat[(data_multi_feat.year >= year_from) & (data_multi_feat.year <= year_to)]
  func_order = ['Min', 'QU_25', 'Median', 'QU_75', 'Mean', 'Max'][::-1]
  data_multi_feat_year['Aggr_func'] = pd.Categorical(data_multi_feat_year['Aggr_func'], categories=func_order, ordered=True)
  data_multi_feat_year = data_multi_feat_year.sort_values(by=['date', 'Aggr_func'])

year_from_dropdown = widgets.Dropdown(
    options=years,
    description='Year From:',
    disabled=False
    )
year_to_dropdown = widgets.Dropdown(
    options=years,
    description='Year To:',
    disabled=False
    )

def on_year_change(change)->None:
    filter_dataframe(year_from_dropdown.value, year_to_dropdown.value)

year_from_dropdown.observe(on_year_change, names='value')
year_to_dropdown.observe(on_year_change, names='value')
display(year_from_dropdown, year_to_dropdown)
filter_dataframe(year_from_dropdown.value, year_to_dropdown.value)

Dropdown(description='Year From:', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, …

Dropdown(description='Year To:', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 20…

In [11]:
fig = px.line(data_multi_feat_year, x="date", y="Surface_Temp", color='Aggr_func', markers=True, title=f'Surface temperature from {year_from_dropdown.value} to {year_to_dropdown.value} years.',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f'], animation_frame="Elevation", range_y=(-50, 50))
fig.show()